In [196]:
import re, pickle, time
from gensim.models.word2vec import Word2Vec
import matplotlib.pyplot as plt
import MeCab
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
from torch import nn, optim
import torch.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [197]:
DATA_PATH="../../data/takken/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
amp=200

In [199]:
def remove_symbol(text):
    '''
    入力されたテキストから句読点などの不要な記号をいくつか削除する。
    '''
    remove_list = [
        ',', '.', '-', '，', '\ufeff', '\u3000', '\n']
    for i, symbol in enumerate(remove_list):
        text = text.replace(symbol, '')
    return text

def add_bos_eos(text):
    return "<BOS> " + text + " <EOS>"

In [198]:
class Vocab():
    
    bos_token=2
    eos_token=3
    
    def __init__(self, min_count=0):
        self.word2id_dict = dict({'<PAD>':0,  '<UNK>': 1, '<BOS>':Vocab.bos_token, '<EOS>':Vocab.eos_token})
        self.id2word_dict = dict({i:word for word, i in self.word2id_dict.items()})
        self.size = 4
        self.min_count = min_count
        self._i = 0  
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._i==self.size:
            self._i=0
            raise StopIteration
        word = self.id2word(self._i)
        self._i += 1
        return word
        
    def add(self, word):
        key = self.word2id_dict.setdefault(word, self.size)
        self.id2word_dict[key] = word
        if key == self.size:
            self.size += 1      
            
    def id2word(self, key):
        return self.id2word_dict.get(key)
    
    def word2id(self, key):
        return self.word2id_dict.get(key)
    

    
#batch_size and datasize are global variables
class DataLoader():
    def __init__(self, inputs):
        self.start_index = 0
        self.inputs=inputs
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.start_index >= datasize:
            self.start_index =0 
            raise StopIteration
        minibatch = self.inputs[self.start_index: self.start_index+batch_size]
        self.start_index += batch_size
        
        minibatch = torch.tensor(minibatch, dtype=torch.long, device=device)
        return minibatch
    
    
    
def trainIters(model, criterion, dataloader, words, print_every=1, plot_every=5,):
    optimizer = optim.Adam(model.parameters())
    plot_losses = []
    for epoch in range(epochs):
        start=time.time()
        train_loss=0
        for batch_id, minibatch in enumerate(dataloader):
            a   = minibatch[:,:-1][:]
            generated = model(a)
            hoge = minibatch[:,1:][:]

            loss = criterion(generated, hoge)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss
            if batch_id % print_every == 0:
                elapsed_sec = time.time() - start
                elapsed_min = int(elapsed_sec / 60)
                elapsed_sec = elapsed_sec - 60 * elapsed_min
                print(
                    'Epoch:{} Batch:{}/{} Loss:{:.4f} Time:{}m{:.1f}s'.format(
                        epoch, batch_id,
                        int(datasize /
                            batch_size),
                        train_loss / (1 + batch_id), elapsed_min,
                        elapsed_sec),
                    end='\r')
                
            if batch_id % plot_every:
                plot_losses.append(loss)
        print()            
        
    return plot_losses

In [215]:
class Rnn(nn.Module):
    def __init__(self, latent_dim, embedding_matrix):
        super(Rnn, self).__init__()
        self.lstm          = nn.LSTM(input_size = embedding_dim, hidden_size = latent_dim)
        self.linear        = nn.Linear(latent_dim, num_tokens)
        embedding_matrix   = torch.from_numpy(embedding_matrix)
        self.embed         = nn.Embedding(num_tokens, embedding_dim=embedding_dim, padding_idx=0, _weight=embedding_matrix)
        
    def forward(self, inputs):
        #以下，maxlenはその値より-1したものを表す
        
        #inputs = (batch, maxlen)
        #inputsはindexに対応
        batch_size     = inputs.shape[0]
        embed          = self.embed(inputs) #(batch, maxlen, embedding_dim)
        
        #make random states
        h=torch.rand(1,batch_size,latent_dim)*amp
        c=torch.rand(1,batch_size,latent_dim)*amp
        states = (h.cuda(), c.cuda())
        
        outputs, hidden = self.lstm(
            embed.permute(1,0,2), states
        ) #(maxlen, batch, latent_dim)
        outputs = self.linear(outputs)  #(maxlen, batch, num_tokens)
        outputs = outputs.permute(1,0,2) #(batch, maxlen, num_tokens)
        outputs = outputs.permute(0,2,1) #(batch, num_tokens, maxlen)
         
        return outputs
    
    
    def sample(self, words, size=20):
        outputs=[]
        
        #describe initial input and initial random states  
        income = torch.tensor(
            [words.word2id("<BOS>")]*size, dtype=torch.long, device=device).unsqueeze(1)  #(size, 1)
        
        h=torch.rand(1, size, latent_dim)*amp
        c=torch.rand(1, size, latent_dim)*amp
        states = (h.cuda(), c.cuda())        
        
        #回す(ほんとは<EOS>で止めるようにしたかった。)
        for i in range(maxlen-1):     
            embed = self.embed(income) #(size, 1, embedding_dim)
            embed = embed.permute(1,0,2)  #(1, size, embedding_dim)
            
            outcome, hidden = self.lstm(embed, states)
            outcome = self.linear(outcome)  #(1, size, num_tokens)
            _, topi = torch.max(outcome, 2)
            outputs.append(topi)
            income = topi.permute(1,0)  #(size, 1)
            
            states = hidden
            
        outputs = torch.cat(outputs, dim=0).permute(1,0)  #(size, max_len)
        
        return outputs            

In [200]:
takken    = pd.read_csv(DATA_PATH+"takken.csv", encoding='utf-8')
mondaishu = pd.read_csv(DATA_PATH+"mondaishu.csv", encoding='utf-8')
nikken    = pd.read_csv(DATA_PATH+"nikken.csv", encoding='utf-8')
legal_mind= pd.read_csv(DATA_PATH+"legal_mind.csv", encoding='utf-8')

takken=takken[["Question","Choice"]]
ocr=pd.concat([mondaishu, nikken, legal_mind], axis=0, ignore_index=True)
ocr = ocr[["Wakati_Question", "Wakati_Choice"]]
ocr.columns = ["Question", "Choice"]

m=MeCab.Tagger("-Owakati")
takken = takken.applymap(remove_symbol)
ocr    = ocr.applymap(remove_symbol)
takken = takken.applymap(m.parse)
takken = pd.concat([takken, ocr], axis=0, ignore_index=True)
takken = takken.applymap(remove_symbol)
takken = takken.applymap(add_bos_eos)

In [201]:
takken=takken["Choice"]
takken

0       <BOS> 建設 業法 による 建設 業 の 許可 を 受け て いる A が 、 建築 請...
1       <BOS> 地主 B が 、 都市 計画 法 の 用途 地域 内 の 所有 地 を 、 駐車...
2       <BOS> 地主 C が 、 その 所有 地 に 自ら マンション を 建設 し た 後 、...
3       <BOS> 農家 D が 、 その 所有 する 農地 を 宅地 に 転用 し 、 全体 を ...
4       <BOS> A が 、 競売 により 取得 し た 複数 の 宅地 を 、 宅地 建物 取引...
5       <BOS> A が 、 土地 区画 整理 事業 により 造成 さ れ た 甲 市 所有 の ...
6       <BOS> A が 、 組合 方式 による 住宅 の 建築 という 名目 で 組合 参加 者...
7       <BOS> A が 、 賃貸 物件 の 複数 の 所有 者 から 一括 し て 借上げ 、 ...
8       <BOS> 建設 会社 A が 、 所有 宅地 を 10 区画 に 分割 し 、 宅地 建物...
9       <BOS> 農業 協同 組合 C が 、 所有 宅地 を 10 区画 に 分割 し 、 倉庫...
10      <BOS> 甲 県 住宅供給公社 D が 、 住宅 を 不 特定 多数 に 継続 し て 販...
11      <BOS> 宅地 建物 取引 士 E が 、 E 名義 で 賃貸 物件 の 媒介 を 反復 ...
12      <BOS> A が 、 その 所有 する 農地 を 区画 割り し て 宅地 に 転用 し ...
13      <BOS> C が 、 その 所有 地 に マンション を 建築 し た うえ で 、 自ら...
14      <BOS> E が 、 その 所有 する 都市 計画 法 の 用途 地域 内 の 農地 を ...
15      <BOS> F が 、 甲 県 から その 所有 する 宅地 の 販売 の 代理 を 依頼 ...
16      <BOS> A の 所有 する オフィス ビル を 賃借 し て いる B が 、 不 特定...
17      <BOS> 

In [202]:
words=Vocab()
for sent in takken:
    for word in sent.split():
        words.add(word)

In [203]:
num_tokens=words.size

In [204]:
word2vec = Word2Vec.load("../wiki_textbook/text_wiki_model")
word2vec_size = 200
embedding_dim = word2vec_size
embedding_matrix = np.random.uniform(low=-0.1, high=0.1, size=(num_tokens, word2vec_size))

In [205]:
unknowns=set()

for i,word in enumerate(words):
    try:
        embedding_matrix[i] = word2vec[word]
    except KeyError:
        unknowns.add(word)
        
embedding_matrix[0] = np.zeros((word2vec_size, ))

embedding_matrix = embedding_matrix.astype('float32')

/home/develop/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [206]:
unknowns.remove("<BOS>")
unknowns.remove("<EOS>")
unknowns.remove("<UNK>")
unknowns.remove("<PAD>")
unknowns

{'1239000',
 '151200',
 '1560000',
 '1660000',
 '1669500',
 '1680000',
 '1684800',
 '1720000',
 '1814400',
 '1944000',
 '2008800',
 '202400',
 '2073600',
 '2106000',
 '2200000',
 '27200',
 '302400',
 '3440000',
 '70200',
 '97200',
 'Discounted',
 'DiscountedCashFlow',
 'アンカーボルト',
 '代払い',
 '備え付けれ',
 '充てれ',
 '切土',
 '却等',
 '換資',
 '立ち入ろ',
 '約定金利',
 '該借',
 '３4'}

In [207]:
datasize = len(takken)

In [208]:
maxlen=0
for sent in takken:
    length= len(sent.split())
    if length>maxlen:
        maxlen=length

In [209]:
#<BOS>付き
maxlen

147

In [210]:
inputs=np.zeros((datasize, maxlen), dtype='int32')

In [211]:
for i,sent in enumerate(takken):
    for j,word in enumerate(sent.split()):
        if word in unknowns:
            word="<UNK>"
        inputs[i][j]=words.word2id(word)

In [213]:
criterion=nn.CrossEntropyLoss(ignore_index=0) 
latent_dim=512
batch_size=32
epochs = 28

In [214]:
embedding_matrix.shape[0]

5003

In [220]:
model=Rnn(latent_dim, embedding_matrix).to(device)
dataloader = DataLoader(inputs)
losses = trainIters(model, criterion, dataloader, words)

Epoch:0 Batch:295/295 Loss:3.8259 Time:0m13.7s
Epoch:1 Batch:295/295 Loss:2.6911 Time:0m13.7s
Epoch:2 Batch:295/295 Loss:2.2790 Time:0m13.7s
Epoch:3 Batch:295/295 Loss:2.0175 Time:0m13.8s
Epoch:4 Batch:295/295 Loss:1.8248 Time:0m13.8s
Epoch:5 Batch:295/295 Loss:1.6679 Time:0m13.8s
Epoch:6 Batch:295/295 Loss:1.5376 Time:0m13.8s
Epoch:7 Batch:295/295 Loss:1.4256 Time:0m13.8s
Epoch:8 Batch:295/295 Loss:1.3265 Time:0m13.8s
Epoch:9 Batch:295/295 Loss:1.2421 Time:0m13.8s
Epoch:10 Batch:295/295 Loss:1.1657 Time:0m13.8s
Epoch:11 Batch:295/295 Loss:1.0998 Time:0m13.8s
Epoch:12 Batch:295/295 Loss:1.0389 Time:0m13.8s
Epoch:13 Batch:295/295 Loss:0.9828 Time:0m13.8s
Epoch:14 Batch:295/295 Loss:0.9323 Time:0m13.8s
Epoch:15 Batch:295/295 Loss:0.8858 Time:0m13.8s
Epoch:16 Batch:295/295 Loss:0.8410 Time:0m13.8s
Epoch:17 Batch:295/295 Loss:0.8002 Time:0m13.8s
Epoch:18 Batch:295/295 Loss:0.7644 Time:0m13.8s
Epoch:19 Batch:295/295 Loss:0.7310 Time:0m13.8s
Epoch:20 Batch:295/295 Loss:0.6998 Time:0m13.8s
Ep

In [221]:
size=50

generated = model.sample(words, size) #(size, maxlen)
kinds = set()
generated = generated.tolist()
for sample in generated:
    sent=""
    
    for index in sample:
        #When <EOS>, finish. This will make sense.
        if index==words.word2id("<EOS>"):
            break       
                
        word = words.id2word(index)
        sent =sent + word + " "
        print(word, end=" ")
    
    print(end="\n\n")
    kinds.add(sent)

A が 、 宅地 建物 取引 業者 で ある B と 売買 契約 を 締結 し た 場合 、 A は 、 B の 手付 金 を 受領 し た 後 に 、 手付 金 と し て 2 500 万 円 を 受領 する こと が できる 。 

A が 、 乙 県知事 に 登録 の 移転 の 申請 とともに 宅地 建物 取引 士 証 の 交付 の 申請 を し た とき は 、 新た な 登録 を 受け て い ない 聞 について は 、 乙 県知事 から 登録 を 消 除 さ れる こと は ない 。 

A が 、 宅地 の 所有 者 D から 当該 宅地 の 売買 契約 を 締結 し た 場合 、 A は 、 保全 措置 を 講じ なけれ ば なら ない 。 

A が 、 宅地 建物 取引 業者 で ある B と 売買 契約 を 締結 し た 場合 、 A は 、 B の 手付 金 を 受領 し た 後 に 、 手付 金 と し て 2 500 万 円 を 受領 する こと が できる 。 

A が 、 宅地 建物 取引 業者 で ある B と 売買 契約 を 締結 し た 場合 、 A は 、 宅地 建物 取引 業法 第 34 条 の 2 の 規定 に 基づく 手付 金 等 の 保全 措置 を 講じる こと なく 、 宅地 建物 取引 業法 第 41 条 の 2 に 規定 する 手付 金 等 の 保全 措置 を 講じる こと なく 手付 金 と し て 2 000 万 円 を 受領 する こと が できる 。 

A が 、 宅地 建物 取引 業法 第 41 条 の 2 に 規定 する 手付 金 等 の 保全 措置 を 講じ た うえ で 、 当該 土地 に関する 権利 の 移転 の 登記 に は 適用 さ れ ない 。 

A が 、 宅地 建物 取引 業者 で ある B と 売買 契約 を 締結 し た 場合 、 A は 、 B の 手付 金 を 受領 し た 後 に 、 手付 金 と し て 2 500 万 円 を 受領 する こと が できる 。 

A が 、 宅地 建物 取引 業者 で ある B と 売買 契約 を 締結 し た 場合 、 A は 、 B の 手付 金 を 受領 し た 後 に 手付 金 全額 の 返還 を 求める こと が できる 

In [222]:
print("different kinds of sentences in %d is %d" % (size, len(kinds)))

different kinds of sentences in 50 is 29
